# Evaluation of Yolo TensorRT Zoo

For [ENGALL-2424](https://aquabyte.atlassian.net/browse/ENGALL-2424) we seek to understand the TensorRT-accelerated inference time (latency) of YoloV3 on the Jetson TX2.

In [16]:
import sys
sys.path.append('/opt/mft-pg')

# The first Yolo zoo didn't have run ID tracking set up right.  Here are the
# MLFlow Run Ids for that zoo
FIRST_YOLO_ZOO_RUN_IDS = (
  '90fafa4fb2194c70ab5e99bf94964587',
  '3771d3357f534c689a31f7278f2fe60e',
  '86d0fbfae7d9432bb15b17113bf3f291',
  '058730c05f8f4dd8a3299fb10dece255',
  '32779f411bbd4c1dafe483ca8a636601',
  'e10cae25a3884f75919c2b6212e4824f',
  '3b33e4f6bc45466c9a1d35ee839a0c75',
  'b18e9351ef4b4da9b788cc135339f457',
  'e235ff91d5c043979f31b8ce92c1e7a5',
  'c114c649c02349389b54810c526bdc7f',
  '95a388e4809d4e24870e4d147140c356',
  '95d86a448acc4d86ba2d3714795453a2',
  'ea283b03af834744a616fa740b4f303a',
  'f231ff83d1dd494fa6269916cc0b1ab3',
  '1ab6658a60a7402ba0398cf445ba22d2',
  '2de468aed4e445a6a352db4b15fd703c',
  '71058625298a4b99a91f5099fd8c7cd4',
  '2176e2dd712949c5bb7e1c25abc3b278',
  '4ff330ca02054a1db9eff79abb06841a',
  '3a10b05bc3344922b0b953f27e61b07b',
  'd7e6b25641e34edd913c66d0e6725a6b',
  'f0af1e307acc4d19b13b89ab3025ccf6',
  '81fe7f4ff374481198a0aa1e94ebe3a1',
  '10a236b73ea64e84b0f7628fa3c6f0f6',
  'e4297a980b7e43ac8099454601895c0a',
  '0053813569944f2ca919993ed3bea4f9',
  '5f7c3d1b1e0c49e1a52b25ec0bf6d316',
  '4e3232e14ae6472a86717be9ce8572a7',
)

print('len(FIRST_YOLO_ZOO_RUN_IDS)', len(FIRST_YOLO_ZOO_RUN_IDS))


len(FIRST_YOLO_ZOO_RUN_IDS) 28


In [5]:

import mlflow
mlflow.set_tracking_uri('/opt/mft-pg/mlruns')
mlflow_df = mlflow.search_runs()
mlflow_df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.trt_engine_load_time_sec,metrics.mean_latency_ms,metrics.total_detection_time_sec,metrics.total_trt_build_time_sec,...,params.min_width,params.max_width,params.training_time_sec,params.convert_to_yolo_format_time_sec,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.project.backend,tags.mlflow.parentRunId,tags.mlflow.project.entryPoint
0,c5ca5cc653e443468a3e7abaa6868d7c,0,RUNNING,file:///opt/mft-pg/mlruns/0/c5ca5cc653e443468a...,2021-05-11 21:43:27.980000+00:00,NaT,NaN,NaN,NaN,NaN,...,None,None,None,None,LOCAL,root,detection/inference/test_detector_zoo.py,None,None,None
1,95f6df588c2a4b96808b0177968f187a,0,RUNNING,file:///opt/mft-pg/mlruns/0/95f6df588c2a4b9680...,2021-05-11 21:35:06.168000+00:00,NaT,NaN,NaN,NaN,NaN,...,None,None,None,None,LOCAL,root,detection/inference/test_detector_zoo.py,None,None,None
2,95d0147b3afc44d399ad3f884a1d453f,0,FINISHED,file:///opt/mft-pg/mlruns/0/95d0147b3afc44d399...,2021-05-11 21:26:41.689000+00:00,2021-05-11 21:32:48.061000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,LOCAL,root,detection/inference/test_detector_zoo.py,None,None,None
3,475e3557f1584c43a29f4977f910ddac,0,FINISHED,file:///opt/mft-pg/mlruns/0/475e3557f1584c43a2...,2021-05-11 21:14:12.443000+00:00,2021-05-11 21:40:30.214000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,LOCAL,root,detection/inference/test_detector_zoo.py,None,None,None
4,29f7fc487bb749d5943f0b6a2190ee44,0,FAILED,file:///opt/mft-pg/mlruns/0/29f7fc487bb749d594...,2021-05-11 21:13:46.209000+00:00,2021-05-11 21:13:53.859000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,LOCAL,root,detection/inference/test_detector_zoo.py,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,77d3b08420734e2bab90621578914d25,0,FINISHED,file:///opt/mft-pg/mlruns/0/77d3b08420734e2bab...,2021-05-05 07:26:51.075000+00:00,2021-05-05 07:27:30.312000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,LOCAL,root,detection/training/darknet/train_yolo_zoo.py,local,8c1927b92cc245b18f579d0ef182ab37,None
184,e929f1181cf849a680b34ba05bf79905,0,FINISHED,file:///opt/mft-pg/mlruns/0/e929f1181cf849a680...,2021-05-05 07:26:51.070000+00:00,2021-05-05 07:27:36.764000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,LOCAL,root,detection/training/darknet/train_yolo_zoo.py,local,8c1927b92cc245b18f579d0ef182ab37,None
185,146981a6c5c544f983016c6b55c5a2e9,0,FINISHED,file:///opt/mft-pg/mlruns/0/146981a6c5c544f983...,2021-05-03 20:38:44.892000+00:00,2021-05-04 16:17:49.280000+00:00,NaN,NaN,NaN,NaN,...,None,None,70734.79656863213,4.6280741691589355,PROJECT,root,/opt/mft-pg,local,None,train_yolo_darknet
186,2cd7d82bab2548458f2521e36e92b876,0,FINISHED,file:///opt/mft-pg/mlruns/0/2cd7d82bab2548458f...,2021-05-03 20:37:56.788000+00:00,2021-05-04 21:06:57.923000+00:00,NaN,NaN,NaN,NaN,...,None,None,88126.00151443481,5.20058012008667,PROJECT,root,/opt/mft-pg,local,None,train_yolo_darknet


In [19]:
import os
import pandas as pd
from mft_utils.bbox2d import BBox2D

stat_df_rows = []

artifact_uris = mlflow_df[mlflow_df['run_id'].isin(FIRST_YOLO_ZOO_RUN_IDS)]['artifact_uri']

for artifact_dir in artifact_uris:
    # Trip the 'file://' thing
    artifact_dir = artifact_dir.replace('file://', '')
    print(artifact_dir)
    
    trt_tx2_engine_path = os.path.join(artifact_dir, 'yolov3.NVIDIA.Tegra.X2.trt')
    trt_rtx_engine_path = os.path.join(artifact_dir, 'yolov3.GeForce.RTX.2070.with.Max-Q.Design.trt')

    trt_tx2_df_path = os.path.join(artifact_dir, 'YoloTRTRunner.NVIDIA.Tegra.X2.detections_df.pkl')
    trt_rtx_df_path = os.path.join(artifact_dir, 'YoloTRTRunner.GeForce.RTX.2070.with.Max-Q.Design.detections_df.pkl')
    
    yolo_config_path = os.path.join(artifact_dir, 'yolov3.cfg')
    
    from mft_utils import misc as mft_misc
    w, h = mft_misc.darknet_get_yolo_input_wh(yolo_config_path)
    
    img_width = w
    
    if os.path.exists(trt_tx2_df_path):
        det_df = pd.read_pickle(trt_tx2_df_path)
        stat_df_rows.append({
            'platform': 'Tegra.X2',
            'img_width': img_width,
            'latencies': det_df['latency_sec'].to_numpy(),
            'trt_engine_size_bytes': os.path.getsize(trt_tx2_engine_path),
            'trt_load_time': det_df['extra'][0]['trt_engine_load_time_sec'],
            'resize_time_sec': det_df['extra'][0]['resize_time_sec'],
        })
    
    if os.path.exists(trt_rtx_df_path):
        det_df = pd.read_pickle(trt_rtx_df_path)
        stat_df_rows.append({
            'platform': 'GeForce.RTX.2070',
            'img_width': img_width,
            'latencies': det_df['latency_sec'].to_numpy(),
            'trt_engine_size_bytes': os.path.getsize(trt_rtx_engine_path),
            'trt_load_time': det_df['extra'][0]['trt_engine_load_time_sec'],
            'resize_time_sec': det_df['extra'][0]['resize_time_sec'],
        })


results_df = pd.DataFrame(stat_df_rows)
print('len(results_df)', len(results_df))
results_df

# for run_id in FIRST_YOLO_ZOO_RUN_IDS:
#     artifact_dir = mlflow_df[mlflow_df['run_id'] == run_id]['artifact_uri']
#     print(artifact_dir)

    
# artifact_dirs = []

# 'YoloTRTRunner.NVIDIA.Tegra.X2.detections_df.pkl'
# 'YoloTRTRunner.GeForce.RTX.2070.with.Max-Q.Design.detections_df.pkl'


# img_width | latency_time_sec | platform

# img_width | trt_engine_size_bytes | platform

# img_width | trt_load_time | platform




/opt/mft-pg/mlruns/0/90fafa4fb2194c70ab5e99bf94964587/artifacts
/opt/mft-pg/mlruns/0/3771d3357f534c689a31f7278f2fe60e/artifacts
/opt/mft-pg/mlruns/0/86d0fbfae7d9432bb15b17113bf3f291/artifacts
/opt/mft-pg/mlruns/0/058730c05f8f4dd8a3299fb10dece255/artifacts
/opt/mft-pg/mlruns/0/32779f411bbd4c1dafe483ca8a636601/artifacts
/opt/mft-pg/mlruns/0/e10cae25a3884f75919c2b6212e4824f/artifacts
/opt/mft-pg/mlruns/0/3b33e4f6bc45466c9a1d35ee839a0c75/artifacts
/opt/mft-pg/mlruns/0/b18e9351ef4b4da9b788cc135339f457/artifacts
/opt/mft-pg/mlruns/0/e235ff91d5c043979f31b8ce92c1e7a5/artifacts
/opt/mft-pg/mlruns/0/c114c649c02349389b54810c526bdc7f/artifacts
/opt/mft-pg/mlruns/0/95a388e4809d4e24870e4d147140c356/artifacts
/opt/mft-pg/mlruns/0/95d86a448acc4d86ba2d3714795453a2/artifacts
/opt/mft-pg/mlruns/0/ea283b03af834744a616fa740b4f303a/artifacts
/opt/mft-pg/mlruns/0/f231ff83d1dd494fa6269916cc0b1ab3/artifacts
/opt/mft-pg/mlruns/0/1ab6658a60a7402ba0398cf445ba22d2/artifacts
/opt/mft-pg/mlruns/0/2de468aed4e445a6a35

,platform,img_width,latencies,trt_engine_size_bytes,trt_load_time,resize_time_sec
0,Tegra.X2,1024,"[0.497387170791626, 0.44397830963134766, 0.427...",313538696,8.796996,0.026302814483642578
1,GeForce.RTX.2070,1024,"[0.03261137008666992, 0.030181407928466797, 0....",141186292,12.739180,0.03009033203125
2,Tegra.X2,992,"[0.48967647552490234, 0.43387937545776367, 0.4...",311297226,8.745568,0.02680063247680664
3,GeForce.RTX.2070,992,"[0.031102657318115234, 0.02866840362548828, 0....",140074044,12.840440,0.002802133560180664
4,Tegra.X2,960,"[0.5131559371948242, 0.40275073051452637, 0.40...",310555688,8.264017,0.02326655387878418
5,GeForce.RTX.2070,960,"[0.030289649963378906, 0.0277249813079834, 0.0...",139118364,12.858822,0.0027816295623779297
6,Tegra.X2,928,"[0.46228551864624023, 0.395660400390625, 0.390...",309841960,8.404645,0.021575450897216797
7,GeForce.RTX.2070,928,"[0.027223587036132812, 0.024672985076904297, 0...",138221612,12.471788,0.0029456615447998047
8,Tegra.X2,896,"[0.4334685802459717, 0.3562314510345459, 0.348...",309163304,8.342298,0.02185988426208496
9,GeForce.RTX.2070,896,"[0.027129650115966797, 0.023804426193237305, 0...",137361340,12.442198,0.002559185028076172


In [21]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
output_notebook()

results_src = ColumnDataSource(results_df)

Loading BokehJS ...

In [37]:
fig = figure(
        title="Inference Latencies for TX2",
        plot_width=950,
        x_axis_label="Latency (milliseconds)",
        y_axis_label="Network input width (pixels)")

for row in results_df.to_dict(orient='records'):
    ys = 1e3 * row['latencies']
    xs = [row['img_width']] * len(ys)
    color = 'blue' if 'Tegra' in row['platform'] else 'orange'
    fig.scatter(xs, ys, fill_alpha=0.25, color=color, legend_label=row['platform'])
#     source=results_src,
#     x='img_width',
#     y='latencies',
#     legend_field='platform')


    
# fig.scatter(
#     source=results_src,
#     x='img_width',
#     y='latencies',
#     legend_field='platform')

show(fig)
#     x, y, marker=typestr,
#             line_color="#6666ee", fill_color="#ee6666", fill_alpha=0.5, size=12)

SyntaxError: invalid syntax (<ipython-input-37-7d71099dec22>, line 5)